# Installing packages section

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml==1.1.6

# Importing packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import joblib
# from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
# from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import configparser
from dateutil.relativedelta import relativedelta
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

# Fetching the datasets

In [ ]:
# from fosforio import snowflake
# from fosforio import get_dataframe

# snowflake.get_connection(connection_name="ME_AD_SALES_CXN")

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [3]:
table_name = 'DF_ALL'
df_all = my_session.sql("select * from {}".format(table_name))
df_all = df_all.to_pandas()
df_all

,AD_DATE,TOTAL_IMPRESSIONS,TOTAL_REVENUE,VIEWABLE_IMPRESSIONS,MEASURABLE_IMPRESSIONS,AD_TYPE,AD_FORMAT,AD_MEDIA_TYPE,DEVICE_TYPE,CITY,POPULATION,CITY_LAT,CITY_LON,LINE_ITEM_GROUP,LINE_ITEM_TYPE,MONETIZATION_CHANNEL,OS_TYPE,SITE_ID,ADVERTISER_ID
0,16-06-2024,122,0.3712,56,122,interactive,Interstitial,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,343,8
1,16-06-2024,170,0.5224,56,170,interactive,in_content,shorts,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,351,8
2,16-06-2024,172,0.6732,56,170,interactive,in_content,survey,tablet,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,android,351,8
3,16-06-2024,120,0.0000,58,120,interactive,banner_customizable,html5,phone,Los Angeles,3820914,34.02,-118.41,non_guaranteed,adsense,sponsorship,mac,347,2639
4,16-06-2024,50,0.0000,58,50,non_interactive,native_rich,content,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in-app_purchases,ios,346,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567286,15-07-2024,28,0.1822,6,22,interactive,rewarded interstitial_customizable,html5,laptop,El Paso,678958,31.85,-106.43,guaranteed,standard,in-app_purchases,ios,346,79
567287,15-07-2024,156,0.2130,6,156,interactive,retargetted,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,windows,349,8
567288,15-07-2024,108,0.2160,6,108,interactive,in_feed,survey,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,sponsorship,windows,350,2634
567289,15-07-2024,122,0.2174,6,122,interactive,banner_customizable,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,343,90


In [4]:
table_name = 'DF_OPT'
df_opt = my_session.sql("select * from {}".format(table_name))
df_opt = df_opt.to_pandas()
df_opt

,AD_DATE,DEVICE_TYPE,CITY,POPULATION,CITY_LAT,CITY_LON,MONETIZATION_CHANNEL,OS_TYPE,SITE_ID,ADVERTISER_ID,TOTAL_REVENUE,TOTAL_IMPRESSIONS,VIEWABLE_IMPRESSIONS,MEASURABLE_IMPRESSIONS,AD_TYPE,AD_FORMAT,AD_MEDIA_TYPE,LINE_ITEM_GROUP,LINE_ITEM_TYPE
0,16-06-2024,phone,Los Angeles,3820914,34.02,-118.41,sponsorship,mac,347,2639,0.0003,0,0,0,interactive,banner_customizable,html5,non_guaranteed,ad_exchange
1,16-06-2024,phone,Los Angeles,3820914,34.02,-118.41,sponsorship,mac,347,2637,0.0003,0,0,0,interactive,Interstitial,shorts,non_guaranteed,ad_exchange
2,16-06-2024,tablet,Chicago,2664452,41.84,-87.68,sponsorship,android,351,2645,0.0016,0,0,0,interactive,in_content,rich_media,non_guaranteed,ad_exchange
3,16-06-2024,phone,New York,8258035,40.66,-73.94,sponsorship,mac,351,2646,0.0009,0,0,0,interactive,in_content,rich_media,non_guaranteed,ad_exchange
4,16-06-2024,tablet,Chicago,2664452,41.84,-87.68,freemium,android,345,2636,0.0005,0,0,0,interactive,rewarded,audio,non_guaranteed,ad_exchange
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567286,15-07-2024,laptop,El Paso,678958,31.85,-106.43,in-app_purchases,ios,346,79,0.1822,28,6,22,interactive,rewarded interstitial_customizable,html5,guaranteed,standard
567287,15-07-2024,laptop,New York,8258035,40.66,-73.94,in_app_advertising,windows,349,8,0.2130,156,6,156,interactive,retargetted,gif,non_guaranteed,ad_exchange
567288,15-07-2024,laptop,New York,8258035,40.66,-73.94,sponsorship,windows,350,2634,0.2160,108,6,108,interactive,in_feed,survey,non_guaranteed,ad_exchange
567289,15-07-2024,laptop,New York,8258035,40.66,-73.94,in_app_advertising,ios,343,90,0.2174,122,6,122,interactive,banner_customizable,gif,non_guaranteed,ad_exchange


In [5]:
# df_all = get_dataframe("DF_ALL")
# df_opt = get_dataframe("DF_OPT")

NameError: name 'get_dataframe' is not defined

In [6]:
df_all_copy = df_all.copy()
df_opt_copy = df_opt.copy()

In [7]:
df_all

,AD_DATE,TOTAL_IMPRESSIONS,TOTAL_REVENUE,VIEWABLE_IMPRESSIONS,MEASURABLE_IMPRESSIONS,AD_TYPE,AD_FORMAT,AD_MEDIA_TYPE,DEVICE_TYPE,CITY,POPULATION,CITY_LAT,CITY_LON,LINE_ITEM_GROUP,LINE_ITEM_TYPE,MONETIZATION_CHANNEL,OS_TYPE,SITE_ID,ADVERTISER_ID
0,16-06-2024,122,0.3712,56,122,interactive,Interstitial,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,343,8
1,16-06-2024,170,0.5224,56,170,interactive,in_content,shorts,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,351,8
2,16-06-2024,172,0.6732,56,170,interactive,in_content,survey,tablet,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,android,351,8
3,16-06-2024,120,0.0000,58,120,interactive,banner_customizable,html5,phone,Los Angeles,3820914,34.02,-118.41,non_guaranteed,adsense,sponsorship,mac,347,2639
4,16-06-2024,50,0.0000,58,50,non_interactive,native_rich,content,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in-app_purchases,ios,346,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567286,15-07-2024,28,0.1822,6,22,interactive,rewarded interstitial_customizable,html5,laptop,El Paso,678958,31.85,-106.43,guaranteed,standard,in-app_purchases,ios,346,79
567287,15-07-2024,156,0.2130,6,156,interactive,retargetted,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,windows,349,8
567288,15-07-2024,108,0.2160,6,108,interactive,in_feed,survey,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,sponsorship,windows,350,2634
567289,15-07-2024,122,0.2174,6,122,interactive,banner_customizable,gif,laptop,New York,8258035,40.66,-73.94,non_guaranteed,ad_exchange,in_app_advertising,ios,343,90


In [8]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567291 entries, 0 to 567290
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   AD_DATE                 567291 non-null  object 
 1   TOTAL_IMPRESSIONS       567291 non-null  int16  
 2   TOTAL_REVENUE           567291 non-null  float64
 3   VIEWABLE_IMPRESSIONS    567291 non-null  int16  
 4   MEASURABLE_IMPRESSIONS  567291 non-null  int16  
 5   AD_TYPE                 567291 non-null  object 
 6   AD_FORMAT               567291 non-null  object 
 7   AD_MEDIA_TYPE           567291 non-null  object 
 8   DEVICE_TYPE             567291 non-null  object 
 9   CITY                    567291 non-null  object 
 10  POPULATION              567291 non-null  int32  
 11  CITY_LAT                567291 non-null  float64
 12  CITY_LON                567291 non-null  float64
 13  LINE_ITEM_GROUP         567291 non-null  object 
 14  LINE_ITEM_TYPE      

In [9]:
df_opt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567291 entries, 0 to 567290
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   AD_DATE                 567291 non-null  object 
 1   DEVICE_TYPE             567291 non-null  object 
 2   CITY                    567291 non-null  object 
 3   POPULATION              567291 non-null  int32  
 4   CITY_LAT                567291 non-null  float64
 5   CITY_LON                567291 non-null  float64
 6   MONETIZATION_CHANNEL    567291 non-null  object 
 7   OS_TYPE                 567291 non-null  object 
 8   SITE_ID                 567291 non-null  int16  
 9   ADVERTISER_ID           567291 non-null  int16  
 10  TOTAL_REVENUE           567291 non-null  float64
 11  TOTAL_IMPRESSIONS       567291 non-null  int16  
 12  VIEWABLE_IMPRESSIONS    567291 non-null  int16  
 13  MEASURABLE_IMPRESSIONS  567291 non-null  int16  
 14  AD_TYPE             

# Predictive Modelling

In [10]:
df_all.AD_DATE = pd.to_datetime(df_all.AD_DATE)

In [11]:
training_data = df_all[df_all.AD_DATE <= '2024-07-05']
testing_data = df_all[df_all.AD_DATE > '2024-07-05']

In [12]:
training_data.shape, testing_data.shape

((374301, 19), (192990, 19))

In [13]:
training_data.drop(['SITE_ID','ADVERTISER_ID',
        'AD_DATE', 'AD_TYPE',   'LINE_ITEM_GROUP',
            'CITY', 'POPULATION', 'CITY_LAT', 'CITY_LON'],axis=1,inplace=True)
testing_data.drop(['SITE_ID','ADVERTISER_ID',
        'AD_DATE', 'AD_TYPE',  'LINE_ITEM_GROUP',
            'CITY', 'POPULATION', 'CITY_LAT', 'CITY_LON'],axis=1,inplace=True)


In [14]:
X_train, y_train = training_data.drop('TOTAL_REVENUE', axis=1), training_data['TOTAL_REVENUE']
X_test, y_test = testing_data.drop('TOTAL_REVENUE', axis=1), testing_data['TOTAL_REVENUE']

In [15]:
X_train

,TOTAL_IMPRESSIONS,VIEWABLE_IMPRESSIONS,MEASURABLE_IMPRESSIONS,AD_FORMAT,AD_MEDIA_TYPE,DEVICE_TYPE,LINE_ITEM_TYPE,MONETIZATION_CHANNEL,OS_TYPE
0,122,56,122,Interstitial,gif,laptop,ad_exchange,in_app_advertising,ios
1,170,56,170,in_content,shorts,laptop,ad_exchange,in_app_advertising,ios
2,172,56,170,in_content,survey,tablet,ad_exchange,in_app_advertising,android
3,120,58,120,banner_customizable,html5,phone,adsense,sponsorship,mac
4,50,58,50,native_rich,content,laptop,ad_exchange,in-app_purchases,ios
...,...,...,...,...,...,...,...,...,...
374296,36,6,36,skyscrapers,html5,tablet,ad_exchange,in_app_advertising,android
374297,18,6,18,Interstitial_customizable,audio,tablet,ad_exchange,in_app_advertising,android
374298,48,6,48,Interstitial_customizable,gif,phone,ad_exchange,in_app_advertising,mac
374299,40,6,40,rewarded_customizable,video,tablet,ad_exchange,in_app_advertising,android


In [16]:
pc_col = ['TOTAL_IMPRESSIONS', 'VIEWABLE_IMPRESSIONS', 'MEASURABLE_IMPRESSIONS']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=2))
        ]), pc_col),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['DEVICE_TYPE', 'LINE_ITEM_TYPE', 'OS_TYPE', 'AD_FORMAT',
       'MONETIZATION_CHANNEL', 'AD_MEDIA_TYPE'])
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [17]:
models = [
    {
        'name': 'GradientBoostingRegressor',
        'regressor': [GradientBoostingRegressor()],
        'regressor__n_estimators': [50, 100, 200],
        'regressor__learning_rate': [0.01, 0.1, 0.2],
        'regressor__max_depth': [3, 5, 7],
        'regressor__subsample': [0.7, 0.8, 0.9],
        'regressor__min_samples_split': [2, 5, 10]
    },
]

In [ ]:
best_estimators = []
for model_params in models:
    model_name = model_params.pop('name')  # Extract the model name
    grid_search = GridSearchCV(pipeline, model_params, cv=3, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_estimator = grid_search.best_estimator_
    best_estimators.append(best_estimator)
    print(f"Training completed for model {model_name}")
    
    # Save the best model
    joblib.dump(best_estimator, f'best_model_{model_name}.pkl')
    print(f"Best model {model_name} saved to best_model_{model_name}.pkl")

In [ ]:
results = []
for estimator in best_estimators:
    y_pred_train = estimator.predict(X_train)
    y_pred_test = estimator.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    results.append({
        'model': estimator.named_steps['regressor'].__class__.__name__,
        'best_params': estimator.named_steps['regressor'].get_params(),
        'mse': mse,
        'r2': r2
    })

results_df = pd.DataFrame(results)
results_df

In [ ]:
df_all_copy

In [ ]:
df_all_copy['PREDICTED_REVENUE'] = best_estimator.predict(df_all_copy)

In [ ]:
df_opt_copy

In [ ]:
df_opt_copy['OPT_REVENUE'] = best_estimator.predict(df_opt_copy)

In [ ]:
df_all_copy_training_data = df_all_copy[df_all_copy.AD_DATE <= '2024-07-05']
df_all_copy_testing_data = df_all_copy[df_all_copy.AD_DATE > '2024-07-05']

In [ ]:
df_opt_copy_training_data = df_opt_copy[df_opt_copy.AD_DATE <= '2024-07-05']
df_opt_copy_testing_data = df_opt_copy[df_opt_copy.AD_DATE > '2024-07-05']

# Pushing Model output to Snowflake

In [ ]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("warehouse")
schema= os.getenv("schema")
database = os.getenv("database")
role =  os.getenv("role")
account =  os.getenv("account")
password= os.getenv("password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

In [ ]:
df_opt_copy

In [ ]:
df_opt_copy.rename(columns={"TOTAL_IMPRESSIONS" : "OPT_TOTAL_IMPRESSIONS","VIEWABLE_IMPRESSIONS" : "OPT_VIEWABLE_IMPRESSIONS","MEASURABLE_IMPRESSIONS" : "OPT_MEASURABLE_IMPRESSIONS","AD_TYPE" : "OPT_AD_TYPE","AD_FORMAT" : "OPT_AD_FORMAT","AD_MEDIA_TYPE" : "OPT_AD_MEDIA_TYPE","LINE_ITEM_GROUP" : "OPT_LINE_ITEM_GROUP","LINE_ITEM_TYPE" : "OPT_LINE_ITEM_TYPE", "MONETIZATION_CHANNEL" : "OPT_MONETIZATION_CHANNEL", "OS_TYPE" : "OPT_OS_TYPE", "DEVICE_TYPE" : "OPT_DEVICE_TYPE"}, inplace=True)

df_opt_copy_testing_data.rename(columns={"TOTAL_IMPRESSIONS" : "OPT_TOTAL_IMPRESSIONS","VIEWABLE_IMPRESSIONS" : "OPT_VIEWABLE_IMPRESSIONS","MEASURABLE_IMPRESSIONS" : "OPT_MEASURABLE_IMPRESSIONS","AD_TYPE" : "OPT_AD_TYPE","AD_FORMAT" : "OPT_AD_FORMAT","AD_MEDIA_TYPE" : "OPT_AD_MEDIA_TYPE","LINE_ITEM_GROUP" : "OPT_LINE_ITEM_GROUP","LINE_ITEM_TYPE" : "OPT_LINE_ITEM_TYPE", "MONETIZATION_CHANNEL" : "OPT_MONETIZATION_CHANNEL", "OS_TYPE" : "OPT_OS_TYPE", "DEVICE_TYPE" : "OPT_DEVICE_TYPE"}, inplace=True)

df_opt_copy_training_data.rename(columns={"TOTAL_IMPRESSIONS" : "OPT_TOTAL_IMPRESSIONS","VIEWABLE_IMPRESSIONS" : "OPT_VIEWABLE_IMPRESSIONS","MEASURABLE_IMPRESSIONS" : "OPT_MEASURABLE_IMPRESSIONS","AD_TYPE" : "OPT_AD_TYPE","AD_FORMAT" : "OPT_AD_FORMAT","AD_MEDIA_TYPE" : "OPT_AD_MEDIA_TYPE","LINE_ITEM_GROUP" : "OPT_LINE_ITEM_GROUP","LINE_ITEM_TYPE" : "OPT_LINE_ITEM_TYPE", "MONETIZATION_CHANNEL" : "OPT_MONETIZATION_CHANNEL", "OS_TYPE" : "OPT_OS_TYPE", "DEVICE_TYPE" : "OPT_DEVICE_TYPE"}, inplace=True)



In [ ]:
df_opt_copy['TOTAL_REVENUE'].sum(), df_opt_copy['OPT_REVENUE'].sum()

In [ ]:
df_all_copy['TOTAL_REVENUE'].sum()

In [ ]:
r2 = r2_score(df_opt_copy['TOTAL_REVENUE'], df_opt_copy['OPT_REVENUE'])
r2

In [ ]:
pd.to_datetime(df_all_copy_training_data.AD_DATE.min()), pd.to_datetime(df_all_copy_training_data.AD_DATE.max())

In [ ]:
pd.to_datetime(df_all_copy_testing_data.AD_DATE.min()), pd.to_datetime(df_all_copy_testing_data.AD_DATE.max())

In [ ]:
# df_snowflake = session.createDataFrame(df_all_copy.values.tolist(),
#         schema = df_all_copy.columns.tolist())

# df_snowflake.write.mode("overwrite").save_as_table("ME_DB.ME_AD_SALES_SCHEMA.FULL_OUTPUT")




# df_snowflake = session.createDataFrame(df_all_copy_training_data.values.tolist(),
#         schema = df_all_copy_training_data.columns.tolist())

# df_snowflake.write.mode("overwrite").save_as_table("ME_DB.ME_AD_SALES_SCHEMA.RAW_TABLE")


# df_snowflake = session.createDataFrame(df_all_copy_testing_data.values.tolist(),
#         schema = df_all_copy_testing_data.columns.tolist())

# df_snowflake.write.mode("overwrite").save_as_table("ME_DB.ME_AD_SALES_SCHEMA.FUTURE_PREDICTION_TABLE")

# df_snowflake = session.createDataFrame(df_opt_copy.values.tolist(),
#         schema = df_opt_copy.columns.tolist())

# df_snowflake.write.mode("overwrite").save_as_table("ME_DB.ME_AD_SALES_SCHEMA.OPT_FULL_OUTPUT")


# df_snowflake = session.createDataFrame(df_opt_copy_training_data.values.tolist(),
#         schema = df_opt_copy_training_data.columns.tolist())

# df_snowflake.write.mode("overwrite").save_as_table("ME_DB.ME_AD_SALES_SCHEMA.OPT_RAW_TABLE")

# df_snowflake = session.createDataFrame(df_opt_copy_testing_data.values.tolist(),
#         schema = df_opt_copy_testing_data.columns.tolist())

# df_snowflake.write.mode("overwrite").save_as_table("ME_DB.ME_AD_SALES_SCHEMA.FUTURE_OPT_TABLE")


In [ ]:
df_all_copy = my_session.createDataFrame(df_all_copy)
df_all_copy.write.mode("overwrite").save_as_table("FULL_OUTPUT")

df_all_copy_training_data = my_session.createDataFrame(df_all_copy_training_data)
df_all_copy_training_data.write.mode("overwrite").save_as_table("RAW_TABLE")

df_all_copy_testing_data = my_session.createDataFrame(df_all_copy_testing_data)
df_all_copy_testing_data.write.mode("overwrite").save_as_table("FUTURE_PREDICTION_TABLE")

In [ ]:
df_opt_copy = my_session.createDataFrame(df_opt_copy)
df_opt_copy.write.mode("overwrite").save_as_table("OPT_FULL_OUTPUT")

df_opt_copy_training_data = my_session.createDataFrame(df_opt_copy_training_data)
df_opt_copy_training_data.write.mode("overwrite").save_as_table("OPT_RAW_TABLE")

df_opt_copy_testing_data = my_session.createDataFrame(df_opt_copy_testing_data)
df_opt_copy_testing_data.write.mode("overwrite").save_as_table("FUTURE_OPT_TABLE")

In [ ]:
#session.write_pandas(df_all_copy, "FULL_OUTPUT", auto_create_table=True,  overwrite=True)
#session.write_pandas(df_all_copy_training_data, "RAW_TABLE", auto_create_table=True,  overwrite=True)
#session.write_pandas(df_all_copy_testing_data, "FUTURE_PREDICTION_TABLE", auto_create_table=True,  overwrite=True)

#session.write_pandas(df_opt_copy, "OPT_FULL_OUTPUT", auto_create_table=True,  overwrite=True)
#session.write_pandas(df_opt_copy_training_data, "OPT_RAW_TABLE", auto_create_table=True,  overwrite=True)
#session.write_pandas(df_opt_copy_testing_data, "FUTURE_OPT_TABLE", auto_create_table=True,  overwrite=True)

# Model Registrartion using fosforml SDK


In [ ]:
#Snowpark lib
from snowflake.snowpark import Session
from fosforio import snowflake
from sklearn.pipeline import Pipeline
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests


In [ ]:
@scoring_func
def score(model, request):

    import json
    payload = request.json["payload"]
    if isinstance(request.json["payload"],str):
        payload_data = eval(payload)
        if isinstance(payload_data['TOTAL_IMPRESSIONS'], int):
                data_json = eval(payload)
                data = pd.DataFrame([data_json])
                prediction = pd.DataFrame(model.predict(data))
                return prediction[0].to_list()[0]
        elif isinstance(payload_data['TOTAL_IMPRESSIONS'], dict):
                data = pd.DataFrame(eval(payload))
                prediction = pd.DataFrame(model.predict(data))
                return prediction[0].tolist()
        elif isinstance(payload_data['TOTAL_IMPRESSIONS'], list):
                data = pd.DataFrame(payload_data)
                prediction = pd.DataFrame(model.predict(data))
                return prediction.tolist()
    return "This method is not allowed"

In [ ]:
import requests

payload = str(X_test.iloc[1:3].to_dict())
req = requests.Request()
req.json = {"payload": payload}
print(score(best_estimator, req))

In [ ]:
req.json

In [ ]:
## registering the model in Fosfor.
model_reg = register_model(best_estimator,
               score, 
               name="Ad_Sales_Prediction_Model", 
               description="Ad_Sales_Prediction_RandomForest_Model",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               init_script="\\n pip install scikit-learn==1.5.1 --no-deps\\n pip install joblib==1.4.2\\n pip install scipy==1.13.1\\n pip install threadpoolctl==3.5.0\\n pip install fosforml==1.0.1\\n pip install fosforio==1.0.1 --no-deps\\n pip install holidays==0.9.9\\n pip install pandas==2.2.2 --no-deps\\n pip install holidays==0.9.9\\n pip install python-dateutil==2.9.0\\n pip install pytz==2024.1\\n pip install six==1.16.0\\n pip install tzdata==2024.1\\n pip install numpy==1.26.4",
               y_true=y_test,
               y_pred=y_pred_test,
               #prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)